In [1]:
import numpy as np
from utils import *
import random

In [2]:
data = open('dinos.txt', 'r').read()
data = data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print(data_size, vocab_size)

19909 27


In [4]:
char_to_ix = {ch:i for i,ch in enumerate(sorted(chars))}
ix_to_char = {i:ch for i,ch in enumerate(sorted(chars))}

In [5]:
def clip(gradients, maxValue):
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
    
    for gradient in [dWax, dWaa, dWya, db, dby]:
        np.clip(gradient, -maxValue, maxValue, gradient)
        
    gradients = {'dWaa': dWaa, 'dWax': dWax, 'dWya': dWya, 'db': db, 'dby':dby}
    return gradients

In [7]:
np.random.seed(3)
dWax = np.random.randn(5,3)*10
dWaa = np.random.randn(5,5)*10
dWya = np.random.randn(2,5)*10
db = np.random.randn(5,1)*10
dby = np.random.randn(2,1)*10

gradients = {'dWaa': dWaa, 'dWax': dWax, 'dWya': dWya, 'db': db, 'dby':dby}
gradients = clip(gradients, 10)
print(gradients['dWaa'][1][2])
print(gradients['dWax'][3][1])
print(gradients['dWya'][1][2])
print(gradients['db'][4])
print(gradients['dby'][1])

10.0
-10.0
0.2971381536101662
[10.]
[8.45833407]


In [9]:
def sample(parameters, char_to_ix, seed):
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    n_x = Wax.shape[1]
    
    x = np.zeros((n_x, 1))
    a_prev = np.zeros((n_a, 1))
    
    indices = []
    
    idx = -1
    counter = 0
    newline_char = char_to_ix['\n']
    while(idx != newline_char and counter != 50):
        a = np.tanh(np.dot(Wax,x)+np.dot(Waa,a_prev)+b)
        z = np.dot(Wya,a) + by
        y = softmax(z)
        
        np.random.randn(counter+seed)
        
        idx = np.random.choice(range(0, vocab_size), p=y.ravel())
        
        indices.append(idx)
        
        x = np.zeros((vocab_size, 1))
        x[idx] = 1
        
        a_prev = a
        seed += 1
        counter += 1
        
    if(counter==50):
        indices.append(char_to_ix['\n'])
        
    return indices

In [14]:
np.random.seed(2)
_, n_a = 20,100

Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)
parameters = {'Wax': Wax, 'Waa': Waa, 'Wya': Wya, 'b': b, 'by': by}

indices = sample(parameters, char_to_ix, 0)
print(indices)

[12, 17, 17, 1, 14, 12, 10, 19, 24, 6, 12, 13, 9, 7, 6, 25, 17, 23, 22, 5, 15, 17, 23, 24, 17, 22, 12, 0]


In [15]:
def optimize(X, Y, a_prev, parameters, learning_rate=0.01):
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    gradients, a = rnn_backward(X, Y, parameters, cache)
    gradients = clip(gradients, 5)
    parameters = update_parameters(parameters, gradients, learning_rate)
    
    return loss, gradients, a[len(X)-1]

In [19]:
np.random.seed(1)
vocab_size, n_a = 27,100
a_prev = np.random.randn(n_a,1)
Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)
parameters = {'Wax': Wax, 'Waa': Waa, 'Wya': Wya, 'b': b, 'by': by}
X = [12,3,5,11,22,3]
Y = [4,14,11,22,25,26]

loss, gradients, a_last = optimize(X, Y, a_prev, parameters, learning_rate=0.01)
print(loss)
print(gradients['dWaa'][1][2])
print(np.argmax(gradients['dWax']))
print(gradients['dWya'][1][2])
print(gradients['db'][4])
print(gradients['dby'][1])
print(a_last[4])

126.50397572165362
0.19470931534720892
93
-0.007773876032003782
[-0.06809825]
[0.01538192]
[-1.]


In [25]:
def model(data, ix_to_char, char_to_ix, num_iterations=35000, n_a=50, dino_names=7, vocab_size=27):
    n_x, n_y = vocab_size, vocab_size
    
    parameters = initialize_parameters(n_a, n_x, n_y)
    
    loss = get_initial_loss(vocab_size, dino_names)
    
    with open('dinos.txt') as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
    
    np.random.seed(0)
    np.random.shuffle(examples)
    
    a_prev = np.zeros((n_a, 1))
    
    for j in range(num_iterations):
        index = j % len(examples)
        X = [None] + [char_to_ix[ch] for ch in examples[index]]
        Y = X[1:] + [char_to_ix['\n']]
        
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters, learning_rate=0.01)
        
        if j % 2000 == 0:
            print('Iteration: %d, Loss: %f' %(j, loss) + '\n')
            seed = 0
            for name in range(dino_names):
                sampled_indices = sample(parameters, char_to_ix, seed)
                print_sample(sampled_indices, ix_to_char)
                seed += 1
            print('\n')
            
    return parameters

In [26]:
parameters = model(data, ix_to_char, char_to_ix)

Iteration: 0, Loss: 23.070858

Cgestojczq
Dka
Wrngnritumochjwvtve
Lwras
Djmkafcjdsu
Quxxnvuzfc
Gkoo


Iteration: 2000, Loss: 23.070858

Mippgorhuirus
Isauzos
Waonysaurus
Pkusaurus
Muauscerosaurus
Saurus
Aurudetgipefbmaalos


Iteration: 4000, Loss: 23.070858

Eyhinisakrnianos
Eglasaurus
Lutosavous
Ulirakosaurus
Oreus
Tmor
Spstrosaurus


Iteration: 6000, Loss: 23.070858

Mproson
Mherocen
Elaeosaurus
Losaustes
Eodelodon
Pnoceopamapor
Zekosaurus


Iteration: 8000, Loss: 23.070858

Lisichus
Raptosaurus
Saurlateragops
Rebboocoras
Rongtodenstosaurus
Terlatops
Itosirrangonocelatops


Iteration: 10000, Loss: 23.070858

Nolechus
Uitia
Dheria
Arrootos
Atataosaurus
Flohusaurus
Otocona


Iteration: 12000, Loss: 23.070858

Zspnoseethomasaurucurayhon
Enlang
Gaknishus
Kecsosaurus
Uxelossratlangapdontedphodes
Cophanosaurus
Eeasaurus


Iteration: 14000, Loss: 23.070858

Thmosaurus
Qinototdesaurus
Sicosaur
Troteosaurus
Traloptilosius
Muricierosaurus
Loryove


Iteration: 16000, Loss: 23.070858

Walodon
Yl